## Монтируем гугл-диск к виртуальной машине

In [ ]:
import sys, os
from google.colab import drive
drive.mount('/content/drive')

## Скачиваем репозиторий *курса*

In [ ]:


%cd /content/drive/MyDrive/
%pwd


os.makedirs ('UClim4HSE', exist_ok = True)

%cd /content/drive/MyDrive/UClim4HSE/
%pwd

if not os.path.isdir('/content/drive/MyDrive/UClim4HSE/UClim4HSE-2025'):
  print ('cloning repository')
  !git clone https://github.com/mvarentsov/UClim4HSE-2025.git
else:
  print ('updating repository')
  %cd /content/drive/MyDrive/UClim4HSE/UClim4HSE-2025/
  ! git pull

%cd /content/drive/MyDrive/UClim4HSE/


In [24]:
sys.path.append('/content/drive/MyDrive/UClim4HSE/UClim4HSE-2025/scripts/')
import meteo_utils
from datetime import date, datetime, timedelta

In [ ]:

station_id = 27518     # идентификатор станции, с которой нужны наблюдения
start = date (2023,1,1) # начало периода загрузки данных
end = date.today()      # конец  периода загрузки данных

meteo_utils.load_weaclim4period (station_id, start, end, 'weaclim_data', adress="http://www.pogodaiklimat.ru/weather.php?")


In [ ]:

df_urb = meteo_utils.read_weaclim_dir ('weaclim_data/27605')
df_rur = meteo_utils.read_weaclim_dir ('weaclim_data/27518')

df_urb = df_urb.rename(columns = {x:'urb_' + x for x in df_urb.columns})
df_rur = df_rur.rename(columns = {x:'rur_' + x for x in df_rur.columns})

df = df_urb.merge (df_rur, left_index=True, right_index=True)
display(df.head())


In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 200

plt.figure()
df['rur_t2m'].plot(color = 'blue')
df['urb_t2m'].plot(color = 'red')
plt.ylabel ('Temperature[K]')

df_sel = df['2023-08':'2023-09']
plt.figure()
df_sel['rur_t2m'].plot(color = 'blue')
df_sel['urb_t2m'].plot(color = 'red')
plt.ylabel ('Temperature[K]')



## Интерактивный график с помощью plotly

In [ ]:

import plotly.express as px

fig = px.line(df, y=['rur_t2m', 'urb_t2m'], title='Simple Time Series Plot') #, labels={'Value': 'Cumulative Value'})
fig.data[0].line.color = 'blue'  # Color for 'rur_t2m'
fig.data[1].line.color = 'orange'  # Color for 'urb_t2m'
fig.show()